<a href="https://colab.research.google.com/github/ptmhoang97/get_dataset_from_fiftyone_coco/blob/main/main_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from google.colab import drive
drive._mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [36]:
# COCO_DATASET_YEAR can be '2014' or '2017'
COCO_DATASET_YEAR = '2017'
# COCO_DATASET_TYPE can be 'train' or 'validation'
COCO_DATASET_TYPE = 'train'
# Check below
COCO_DATASET_CLASS = 'car'
# scale of train dataset, "test dataset = 1 - train dataset"
scale_train = 0.8

#* COCO_DATASET_CLASS
person,
bicycle,
car,
motorcycle,
airplane,
bus,
train,
truck,
boat,
traffic light,
fire hydrant,
stop sign,
parking meter,
bench,
bird,
cat,
dog,
horse,
sheep,
cow,
elephant,
bear,
zebra,
giraffe,
backpack,
umbrella,
handbag,
tie,
suitcase,
frisbee,
skis,
snowboard,
sports ball,
kite,
baseball bat,
baseball glove,
skateboard,
surfboard,
tennis racket,
bottle,
wine glass,
cup,
fork,
knife,
spoon,
bowl,
banana,
apple,
sandwich,
orange,
broccoli,
carrot,
hot dog,
pizza,
donut,
cake,
chair,
couch,
potted plant,
bed,
dining table,
toilet,
tv,
laptop,
mouse,
remote,
keyboard,
cell phone,
microwave,
oven,
toaster,
sink,
refrigerator,
book,
clock,
vase,
scissors,
teddy bear,
hair drier,
toothbrush

#1. Install FiftyOne

In [37]:
!pip install fiftyone

In [38]:
!fiftyone --version

FiftyOne v0.14.3, Voxel51, Inc.


In [39]:
!pip install "opencv-python-headless<4.3"

In [40]:
import fiftyone

#2. Download images and generate annotation files

In [41]:
import fiftyone.zoo as foz

In [42]:
import os
print(os.name)

posix


In [43]:
dataset_name_raw = 'coco{}_{}_{}'.format(COCO_DATASET_YEAR,COCO_DATASET_TYPE,COCO_DATASET_CLASS)

In [44]:
paths = {
    'COCO_DATASET_PATH': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR)),
    'RESULT_PATH': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR), dataset_name_raw),
 }

In [45]:
dataset = foz.load_zoo_dataset(name = 'coco-{}'.format(COCO_DATASET_YEAR),\
                               classes = COCO_DATASET_CLASS,\
                               split = COCO_DATASET_TYPE,\
                               dataset_dir = paths['COCO_DATASET_PATH'])

Found annotations at 'fiftyone/coco-2017/raw/instances_train2017.json'
3664 images found; downloading the remaining 8587
 100% |████████████████| 8587/8587 [13.3m elapsed, 0s remaining, 11.0 images/s]      
Writing annotations for 14714 downloaded samples to 'fiftyone/coco-2017/train/labels.json'
Dataset info written to 'fiftyone/coco-2017/info.json'
Loading existing dataset 'coco-2017-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [46]:
!mkdir {paths['RESULT_PATH']}

In [47]:
from pycocotools.coco import COCO
import os
import shutil
from tqdm import tqdm
# import skimage.io as io
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageDraw

In [48]:
resultDir=paths['RESULT_PATH']
srcImageDir = paths['COCO_DATASET_PATH'] + '/{}/{}'.format(COCO_DATASET_TYPE,'data')
srcAnnoDir = paths['COCO_DATASET_PATH'] + '/{}'.format('raw')
datasets_list=[]
if COCO_DATASET_TYPE == 'validation':
    temp = 'val' + COCO_DATASET_YEAR
    datasets_list = [temp]
elif COCO_DATASET_TYPE == 'train':
    temp = 'train' + COCO_DATASET_YEAR
    datasets_list = [temp]
classes_names = [COCO_DATASET_CLASS]
print(datasets_list)
print(classes_names)
print("resultDir: " + str(resultDir))
print("srcImageDir: " + str(srcImageDir))
print("srcAnnoDir: " + str(srcAnnoDir))

['train2017']
['car']
resultDir: fiftyone/coco-2017/coco2017_train_car
srcImageDir: fiftyone/coco-2017/train/data
srcAnnoDir: fiftyone/coco-2017/raw


In [49]:
headstr = """\
<annotation>
    <folder>VOC</folder>
    <filename>%s</filename>
    <source>
        <database>My Database</database>
        <annotation>COCO</annotation>
        <image>flickr</image>
        <flickrid>NULL</flickrid>
    </source>
    <owner>
        <flickrid>NULL</flickrid>
        <name>company</name>
    </owner>
    <size>
        <width>%d</width>
        <height>%d</height>
        <depth>%d</depth>
    </size>
    <segmented>0</segmented>
"""
objstr = """\
    <object>
        <name>%s</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>%d</xmin>
            <ymin>%d</ymin>
            <xmax>%d</xmax>
            <ymax>%d</ymax>
        </bndbox>
    </object>
"""
 
tailstr = '''\
</annotation>
'''
 
#if the dir is not exists,make it,else delete it
def id2name(coco):
    classes=dict()
    for cls in coco.dataset['categories']:
        classes[cls['id']]=cls['name']
    return classes
 
def write_xml(anno_path,head, objs, tail):
    f = open(anno_path, "w")
    f.write(head)
    for obj in objs:
        f.write(objstr%(obj[0],obj[1],obj[2],obj[3],obj[4]))
    f.write(tail)
 
def save_annotations_and_imgs(coco,dataset,filename,objs):
    #eg:COCO_train2014_000000196610.jpg-->COCO_train2014_000000196610.xml
    result_anno_path = '{}/{}{}'.format(resultDir,filename[:-3],'xml')
    #print(result_anno_path)
    src_img_path = '{}/{}'.format(srcImageDir,filename)
    #print(anno_path)
    #print(src_img_path)
    dst_imgpath = '{}/{}'.format(resultDir,filename)
    #print(dst_imgpath)
    img=cv2.imread(src_img_path)
    #if (img.shape[2] == 1):
    #    print(filename + " not a RGB image")
     #   return
    shutil.copy(src_img_path, dst_imgpath)
 
    head=headstr % (filename, img.shape[1], img.shape[0], img.shape[2])
    tail = tailstr
    write_xml(result_anno_path,head, objs, tail)
 
 
def showimg(coco,dataset,img,classes,cls_id,show=True):
    #global dataDir
    path = '{}/{}'.format(srcImageDir,img['file_name'])
    #print(path)
    I=Image.open(path)
    #By id, get comment information
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=cls_id, iscrowd=None)
    # print(annIds)
    anns = coco.loadAnns(annIds)
    # print(anns)
    # coco.showAnns(anns)
    objs = []
    for ann in anns:
        class_name=classes[ann['category_id']]
        if class_name in classes_names:
            #print(class_name)
            if 'bbox' in ann:
                bbox=ann['bbox']
                xmin = int(bbox[0])
                ymin = int(bbox[1])
                xmax = int(bbox[2] + bbox[0])
                ymax = int(bbox[3] + bbox[1])
                obj = [class_name, xmin, ymin, xmax, ymax]
                objs.append(obj)
                draw = ImageDraw.Draw(I)
                draw.rectangle([xmin, ymin, xmax, ymax])
    if show:
        plt.figure()
        plt.axis('off')
        plt.imshow(I)
        plt.show()
 
    return objs

for dataset in datasets_list:
    #./COCO/annotations/instances_train2014.json
    annFile='{}/instances_{}.json'.format(srcAnnoDir,dataset)
    #print(annFile)
    #COCO API for initializing annotated data
    coco = COCO(annFile)
 
    #show all classes in coco
    classes = id2name(coco)
    #print(classes)
    #[1, 2, 3, 4, 6, 8]
    classes_ids = coco.getCatIds(catNms=classes_names)
    #print(classes_ids)
    for cls in classes_names:
        #Get ID number of this class
        cls_id=coco.getCatIds(catNms=[cls])
        img_ids=coco.getImgIds(catIds=cls_id)
        #print(cls,len(img_ids))
        # imgIds=img_ids[0:10]
        for imgId in tqdm(img_ids):
            img = coco.loadImgs(imgId)[0]
            filename = img['file_name']
            # print(filename)
            objs=showimg(coco, dataset, img, classes,classes_ids,show=False)
            #print(objs)
            save_annotations_and_imgs(coco, dataset, filename, objs)

loading annotations into memory...
Done (t=18.33s)
creating index...
index created!


100%|██████████| 12251/12251 [03:17<00:00, 61.93it/s]


#3. Split VOC dataset

In [50]:
dataset_name_VOC = "{0}_VOC".format(dataset_name_raw)
dataset_name_VOC_zip = "{0}.zip".format(dataset_name_VOC)

In [51]:
file_raw_location = paths['RESULT_PATH']

In [52]:
file_VOC_location = "/content/{0}".format(dataset_name_VOC)
file_VOC_location_zip = "/content/{0}.zip".format(dataset_name_VOC)

In [53]:
import shutil

shutil.copytree(file_raw_location, file_VOC_location)

'/content/coco2017_train_car_VOC'

In [54]:
from PIL import Image
import os
import fnmatch

def convert_png_to_jpg(file_location):
    pattern = '*.png'
    image_path = []
    for path in fnmatch.filter(os.listdir(file_location), pattern):
        full_path = os.path.join(file_location, path)
        if os.path.isfile(full_path):
            image_path.append(full_path)
    
    if not image_path:
        print("No png images")
    else:
        print("Converting png to jpg ...")
        for path in image_path:
            path_png = '{}'.format(path)
            path_jpg = path_png.replace(".png",".jpg")
            
            # Convert the link with '\' to '\\' so that it can pass to Image.open() 
            path_png = path_png.encode('unicode-escape').decode()
            path_jpg = path_jpg.encode('unicode-escape').decode()

            img_png = Image.open(path_png)
            img_png_rbg = img_png.convert('RGB')
            img_png_rbg.save(path_jpg)
        print("Done converting!")

In [55]:
import os
import fnmatch

def delete_all_file(file_location,type_file):
    pattern = '*.{0}'.format(type_file)
    file_path = []
    for path in fnmatch.filter(os.listdir(file_location), pattern):
        full_path = os.path.join(file_location, path)
        if os.path.isfile(full_path):
            file_path.append(full_path)

    if not file_path:
        print("No files with type: " + str(type_file))
    else:
        print("Deleting all files with type: " + str(type_file))
        for path in file_path:
            if path.endswith(type_file):
                os.remove(path)
        print("Done deleting!")

In [56]:
import os
import fnmatch
import shutil

def returnNotMatches(a, b):
    return [[x for x in a if x not in b], [x for x in b if x not in a]]

def filter_files(file_location,type_file):
    pattern = "*.{}".format(type_file)
    paths = []
    for path in fnmatch.filter(os.listdir(file_location), pattern):
        path = path[:-4]
        paths.append(path)
    return paths

def check_pair_dataset(file_location,type_file_pic,type_file_anno):
    path_jpg = filter_files(file_location,type_file_pic)
    path_txt = filter_files(file_location,type_file_anno)

    for path in path_txt:
        if path == "classes":
            path_txt.remove("classes")
    print("Checking pair dataset ...")
    if returnNotMatches(path_jpg,path_txt)[0]:
        raise Exception("Files " + type_file_pic + ": " + str(returnNotMatches(path_jpg,path_txt)[0]) + " miss " + type_file_anno + " files")
    elif returnNotMatches(path_jpg,path_txt)[1]:
        raise Exception("Files " + type_file_anno + ": " + str(returnNotMatches(path_jpg,path_txt)[1]) + " miss " + type_file_pic + " files")
    else:
        pass
    print("Done checking! No mismtach!")

In [57]:
from PIL import Image
import os
import fnmatch

def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    with open(file_name, 'w') as f:
        f.writelines(lines)
        f.close()

def rename_file_with_order(file_location,name_file,start_count_number,type_file):
    pattern = "*.{}".format(type_file)
    file_path = []
    for path in fnmatch.filter(os.listdir(file_location), pattern):
        full_path = os.path.join(file_location, path)
        if os.path.isfile(full_path):
            file_path.append(full_path)

    temp_count = start_count_number
    
    for path in file_path:
        name = "{0}_{1}".format(name_file,temp_count)
        start = path.rfind("/")
        end = path.rfind(".")

        start_part = path[:start+1]
        end_part = path[end:]
        
        old_name = path
        new_name = start_part + name + end_part

        if type_file == "xml":
            replace_text = "    <filename>{0}.jpg</filename>\n".format(name)
            replace_line(old_name,2,replace_text)
        
        os.rename(old_name,new_name)
        temp_count+=1

def rename_picFile_and_annoFile_with_order(file_location,name_file,start_count_number,type_pic_file,type_anno_file):
    print("Renaming files ...")

    # Rename pic file
    rename_file_with_order(file_location,name_file,start_count_number,type_pic_file)  
    # Rename anno file
    rename_file_with_order(file_location,name_file,start_count_number,type_anno_file)

In [58]:
import os, sys
import shutil
import re
import tqdm

import math

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def split_dataset(file_location,scale_train):
    print("Splitting dataset ...")

    paths = {
        'TRAIN_PATH': os.path.join(file_location,"train"),
        'TEST_PATH': os.path.join(file_location,"test"),
    }
            
    files = os.listdir(file_location)
    dataset_files = []
    for file_name in files:
        if file_name.endswith("jpg") or file_name.endswith("xml"):
            dataset_files.append(file_name)
        else:
            pass
    
    for path in paths.values():
        if not os.path.exists(path):
            !mkdir {path}
        else:
            !rm -rf {path}
            !mkdir {path}

    total_dataset = len(dataset_files) 
    num_train_dataset = round_up_to_even(total_dataset*scale_train)

    print("total_dataset: " + str(total_dataset))
    print("scale_train: " + str(scale_train))
    print("num_train_dataset: " + str(num_train_dataset))
    print("num_test_dataset: " + str(total_dataset - num_train_dataset))

    # Sort to split files with order from small to big
    print("before sort: " + str(dataset_files))
    dataset_files.sort(key=lambda f: int(re.sub('\D', '', f)))
    print("after sort: " + str(dataset_files))

    # Cut file to train and test folder
    for idx,files in enumerate(dataset_files):
        # print(files)
        if idx < num_train_dataset:
            shutil.move(os.path.join(file_location,files),paths['TRAIN_PATH'])
        else:
            shutil.move(os.path.join(file_location,files),paths['TEST_PATH'])

    # Re-count number of files in train and test folder after cut
    train_files = os.listdir(paths['TRAIN_PATH'])
    test_files = os.listdir(paths['TEST_PATH'])

    print("files in train folder after cut: " + str(len(train_files)))
    print("files in test folder after cut: " + str(len(test_files)))

In [59]:
convert_png_to_jpg(file_VOC_location)

No png images


In [60]:
delete_all_file(file_VOC_location,"png")

No files with type: png


In [61]:
check_pair_dataset(file_VOC_location,"jpg","xml")

Checking pair dataset ...
Done checking! No mismtach!


In [62]:
file_name = COCO_DATASET_CLASS + "_COCO"

In [63]:
rename_picFile_and_annoFile_with_order(file_VOC_location,file_name, 1 ,"jpg","xml")

Renaming files ...


In [64]:
split_dataset(file_VOC_location,scale_train)

Splitting dataset ...
total_dataset: 24502
scale_train: 0.8
num_train_dataset: 19602
num_test_dataset: 4900
before sort: ['car_COCO_10683.jpg', 'car_COCO_5199.jpg', 'car_COCO_8557.jpg', 'car_COCO_3239.jpg', 'car_COCO_8305.jpg', 'car_COCO_1521.xml', 'car_COCO_6348.jpg', 'car_COCO_9903.jpg', 'car_COCO_9702.xml', 'car_COCO_7766.xml', 'car_COCO_11479.xml', 'car_COCO_10909.xml', 'car_COCO_7175.xml', 'car_COCO_1141.xml', 'car_COCO_11966.jpg', 'car_COCO_6096.jpg', 'car_COCO_11362.jpg', 'car_COCO_7769.jpg', 'car_COCO_2548.xml', 'car_COCO_4332.xml', 'car_COCO_5066.jpg', 'car_COCO_3176.xml', 'car_COCO_7034.jpg', 'car_COCO_4285.xml', 'car_COCO_1853.jpg', 'car_COCO_9189.xml', 'car_COCO_589.jpg', 'car_COCO_3315.jpg', 'car_COCO_7211.xml', 'car_COCO_8798.xml', 'car_COCO_4895.jpg', 'car_COCO_8585.xml', 'car_COCO_10982.jpg', 'car_COCO_10909.jpg', 'car_COCO_1771.jpg', 'car_COCO_7684.jpg', 'car_COCO_4946.jpg', 'car_COCO_8940.jpg', 'car_COCO_5796.xml', 'car_COCO_5655.xml', 'car_COCO_7793.jpg', 'car_COCO_1

# 4. Zip and copy dataset to GG drive

In [65]:
!zip -r {dataset_name_VOC_zip} {dataset_name_VOC}

Streaming output truncated to the last 5000 lines.
  adding: coco2017_train_car_VOC/train/car_COCO_5534.jpg (deflated 0%)
  adding: coco2017_train_car_VOC/train/car_COCO_9072.jpg (deflated 0%)
  adding: coco2017_train_car_VOC/train/car_COCO_3078.jpg (deflated 5%)
  adding: coco2017_train_car_VOC/train/car_COCO_3928.xml (deflated 74%)
  adding: coco2017_train_car_VOC/train/car_COCO_7556.xml (deflated 60%)
  adding: coco2017_train_car_VOC/train/car_COCO_1263.jpg (deflated 0%)
  adding: coco2017_train_car_VOC/train/car_COCO_2592.jpg (deflated 0%)
  adding: coco2017_train_car_VOC/train/car_COCO_4284.jpg (deflated 0%)
  adding: coco2017_train_car_VOC/train/car_COCO_7801.jpg (deflated 0%)
  adding: coco2017_train_car_VOC/train/car_COCO_5587.jpg (deflated 0%)
  adding: coco2017_train_car_VOC/train/car_COCO_6660.jpg (deflated 0%)
  adding: coco2017_train_car_VOC/train/car_COCO_6377.xml (deflated 69%)
  adding: coco2017_train_car_VOC/train/car_COCO_9623.jpg (deflated 0%)
  adding: coco2017_trai

In [66]:
!cp {dataset_name_VOC_zip} "/content/gdrive/MyDrive/_dataset"